In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import matplotlib.axes
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Activation
import math
import tensorflow as tf
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
import datetime 

OPEN-> 
CLOSE->
HIGH->
LOW->
PREV.CLOSE->
LTP->
CLOSE->
VWAP-> It's a trading benchmark that represents the average price a security has traded at throughout the day, based on both volume and price.
52W H->
52W L->	
VOLUME->
VALUE->
NO OF TRADES->


In [2]:
read_data_dir="D:\\programs\\ads_flask\\logres_op\\data"
read_model_dir="D:\\programs\\ads_flask\\logres_op\\model"
res_save_dir="D:\\programs\\ads_flask\\logres_op\\residuals"

In [3]:
# get all paths and filenames of files in a dir
ls=[]
files=[]
for root, dirs, files in os.walk(read_data_dir, topdown=False):
   for name in files:
      ls.append(os.path.join(root, name))
for i in range(len(files)):
   files[i]=files[i].replace(".csv","")

In [4]:
ls_m=[]
files_m=[]
for root, dirs, files_m in os.walk(read_data_dir, topdown=False):
   for name in files_m:
      ls_m.append(os.path.join(root, name))
for i in range(len(files_m)):
   files_m[i]=files_m[i].replace(".h5","")

In [1]:
def plt_res(yac,ypr,option,index):
    '''plots a residual graph b/w yactual and ypredicted using the sns.residplot function '''
    for i in range(yac.shape[0]):
        sns.residplot(y=yac[i],x=ypr[i],color='b')
        plt.title(files[index]+" log return residuals on the "+option+" dataset")
        plt.xlabel("predicted log returns")
        plt.ylabel("actual log returns")
    plt.savefig(res_save_dir+"\\"+files[index]+option+".png")


In [6]:
# generating residual graphs for a set of stocks
for h in range(len(ls)):
    dfr=pd.read_csv(ls[h])
    dfr.drop("Unnamed: 0",axis=1,inplace=True)
    time=90# time range to predict
    data=[]
    for i in range(time,dfr["OPEN "].size):
        data.append(np.array(dfr["logret"][i-time:i]))
    data=pd.DataFrame(data)
    dfr["OPEN "].size,data.shape   
    split=math.ceil(data.shape[0]*0.8)
    x_train=[]
    y_train=[]
    for i in range(split-1):
        x_train.append(data.T[i])
        y_train.append(data.T[i+1])
    x_test=[]
    y_test=[]
    for i in range(split,data.shape[0]-1):
        x_test.append(data.T[i])
        y_test.append(data.T[i+1])
    x_train=np.array(x_train).reshape(split-1,time,1)
    x_test=np.array(x_test).reshape(data[0].size-split-1,time,1)
    y_train=np.array(y_train).reshape(split-1,time,1)
    y_test=np.array(y_test).reshape(data[0].size-split-1,time,1)
    x_train.shape,x_test.shape,y_train.shape,y_test.shape
    rl=tf.keras.models.load_model("D:\\programs\\Applied data Science\\proj\\logres_op\\model\\"+files[h]+".h5")
    op_te=rl.predict(x_test)
    op_tr=rl.predict(x_train)
    plt_res(y_test,op_te,"test",h)
    plt_res(y_train,op_tr,"train",h)   

10/60 [====>.........................] - ETA: 12s

KeyboardInterrupt: 